# Regression Used Cars Dataset

https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data 


#### **Dependency**

In [1]:
#! pip install lightgbm

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from itertools import combinations  # For creating combinations of elements
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
# import lightgbm as lgb 
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn import preprocessing

In [2]:
from itertools import combinations  # For creating combinations of elements
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from scipy.stats import randint as sp_randint
from scipy.stats import uniform

#### **Utility Function**

In [3]:
def reduce_mem_usage(df, verbose=0):
    """
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    # Calculate the initial memory usage of the DataFrame
    start_mem = df.memory_usage().sum() / 1024**2

    # 🔄 Iterate through each column in the DataFrame
    for col in df.columns:
        col_type = df[col].dtype
        # Check if the column's data type is not 'object' (i.e., numeric)
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            # Check if the column's data type is an integer
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # Check if the column's data type is a float
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)
    # ℹ️ Provide memory optimization information if 'verbose' is True
    if verbose:
        print(f"Memory usage of dataframe is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        print(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        print(f"Decreased by {decrease:.2f}%")

    # Return the DataFrame with optimized memory usage
    return df

#### **Load Data**

load csv inside data folder

In [4]:
df = pd.read_csv("kaggle/input/vehicles/vehicles.csv")
df=reduce_mem_usage(df,verbose=True)
df.shape
df[0:4]

Memory usage of dataframe is 84.68 MB
Memory usage after optimization is: 76.54 MB
Decreased by 9.62%


,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN


#### **Split dataset** 

In [5]:
df_train, df_test = train_test_split(df, test_size=0.2)
print(df_train.shape, df_test.shape)

(341504, 26) (85376, 26)


#### **Data Function**

data preperation

feature engineering

In [6]:
def data_preparation(df:pd.DataFrame)->pd.DataFrame:
    """"
    make data valid
    """
    #There are too many categories for manufacturer and region,to reduce the risk of overfitting,we should cut the number of categories.
    mf = df['manufacturer'].value_counts()
    df['manufacturer'] = df['manufacturer'].apply(lambda s: s if str(s) in mf[:30] else 'others')

    rg = df['region'].value_counts()
    df['region'] = df['region'].apply(lambda s: s if str(s) in rg[:100] else 'others')
  
    #Create df1, get the average value of each year with deleting NAN and outliers, then assign it to the NAN in df, 
    #and then delete the outliers in df to get the latest data set
    df1 = df.copy()
    odometer_data = df1['odometer']
    odometer_data_cleaned = odometer_data.dropna()
    kmeans = KMeans(n_clusters=4)
    kmeans.fit(odometer_data_cleaned.values.reshape(-1, 1))
    outliers = odometer_data_cleaned[kmeans.labels_ == 1]
    df1 = df1[~df1['odometer'].isin(outliers)]

    mean_odometer = df1.groupby('year')['odometer'].mean()
    mean_odometer_dict = mean_odometer.to_dict()

    df['odometer'] = df.apply(lambda row: mean_odometer_dict[row['year']] if pd.isnull(row['odometer']) and row['year'] in mean_odometer_dict else row['odometer'], axis=1)

    #The rest in odometer does not match the NAN of yaer,replaced using the mean
    mean_avg_odometer = mean_odometer.mean()
    df['odometer'] = df['odometer'].fillna(mean_avg_odometer)

    odometer_data_new = df['odometer']
    kmeans = KMeans(n_clusters=4)
    kmeans.fit(odometer_data_new.values.reshape(-1, 1))
    outliers = odometer_data_new[kmeans.labels_ == 1]
    df = df[~df['odometer'].isin(outliers)]

    #We just keep 10%(about $500)~99.9% of the 'price' value,kick off the ridiculious price.
    df = df[(df['price']<df['price'].quantile(0.999)) & (df['price']>df['price'].quantile(0.1))]

    #use linear regression to prediect values for null in year col.
    features = ["price",'odometer']
    df_complete = df.dropna(subset=["year"] + features)
    X_train = df_complete[features]
    y_train = df_complete["year"]
    regression_model_year = LinearRegression()
    regression_model_year.fit(X_train, y_train)
    X_missing = df[df["year"].isnull()][features]
    if X_missing.shape[0] > 0:
        predicted_year = regression_model_year.predict(X_missing)
        df.loc[df["year"].isnull(), "year"] = predicted_year.round().astype(int)
   

    #Fill null values using the modes of "manufacturer", "cylinders", 'fuel',"title_status", "transmission","drive", "type", "paint_color", "lat", "long"
    columns_to_fillna = ['manufacturer', 'cylinders', 'fuel', 'title_status', 'transmission',
             'drive', 'type', 'paint_color', 'lat', 'long','posting_date']
    modes = df[columns_to_fillna].mode().iloc[0]
    df.loc[:, columns_to_fillna] = df.loc[:, columns_to_fillna].fillna(value=modes)

    #for "condition" col: Fill NAN with randomly selected data from that column to manipulate the more possible results.
    # Calculate the probability of each value (excluding NaN)
    prob_values = df[df['condition'].notna()]['condition'].value_counts(normalize=True)
    condition_options = df["condition"].unique()
    df.loc[df['condition'].isna(), 'condition'] = np.random.choice(prob_values.index, size=df['condition'].isna().sum(), p=prob_values.values)
    df['condition'] = df['condition'].astype(str)

    # Calculate the probability of each value (excluding 'other')
    prob_values = df[df['cylinders'] != 'other']['cylinders'].value_counts(normalize=True)
    # Assign 'other' values based on the probability
    df.loc[df['cylinders'] == 'other', 'cylinders'] = np.random.choice(prob_values.index, size=df['cylinders'].eq('other').sum(), p=prob_values.values)
    # Extract numeric values from the 'cylinder' column
    df['cylinders'] = df['cylinders'].str.extract('(\d+)').astype(int)


    y = df['price']
    x =df.drop(columns=['price'])

    return x,y

def feature_engineering(df:pd.DataFrame)->pd.DataFrame:
    """"
    select useful feature
    """
    #vehicle_age=posting_year - year, get the usage time of this vehicle
    df["posting_date"] = pd.to_datetime(df["posting_date"], utc=True)
    df["posting_year"] = df["posting_date"].dt.year
    df["vehicle_age"] = df["posting_year"] - df["year"]
    
    #Remove meaningless columns
    #modify
    df.drop(['id','url','region','region_url','model','title_status',
         'image_url','VIN','size','county','description','state',
         'lat','long','posting_date','posting_year'], 
        axis=1, inplace=True)
    
    return df

def encoding(df:pd.DataFrame,encoders=None)->pd.DataFrame:
    cols = df.columns.values.tolist()
    numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    categorical_cols = []
    for col in cols:
        if df[col].dtype in numerics: continue
        categorical_cols.append(col)

    if(encoders==None):
        encoders=[]
        for col in categorical_cols:
            le = preprocessing.LabelEncoder()
            le.fit(list(df[col].astype(str).values))
            df[col] = le.transform(list(df[col].astype(str).values))
            encoders.append(le)
    else:
        id=0
        for col in categorical_cols:
            le = encoders[id]
            df[col] = le.transform(list(df[col].astype(str).values))
            id=id+1

    return df,encoders



def data_preprocessing(df:pd.DataFrame ,datasetType="train", labelEncoders=None)->pd.DataFrame:
    x , y =data_preparation(df)
    x=feature_engineering(x)
    encoders=labelEncoders
    if(datasetType=="train"):
        x,encoders=encoding(x,encoders=encoders)
    if(datasetType=="test"):
        if(encoders==None):
            print("Please give a encoder")
            raise RuntimeError
        x,_ = encoding(x,encoders)
    return x,y,encoders

In [7]:
df_train_x,df_train_y,encoders = data_preprocessing(df_train,datasetType="train")
df_test_x, df_test_y,_ = data_preprocessing(df_test,datasetType="test",labelEncoders=encoders)
print(df_train_x.isna().sum())
print(df['type'].unique())
print(df_train_x.shape, df_train_y.shape)

year            0
manufacturer    0
condition       0
cylinders       0
fuel            0
odometer        0
transmission    0
drive           0
type            0
paint_color     0
vehicle_age     0
dtype: int64
[nan 'pickup' 'truck' 'other' 'coupe' 'SUV' 'hatchback' 'mini-van' 'sedan'
 'offroad' 'bus' 'van' 'convertible' 'wagon']
(306589, 11) (306589,)


In [8]:
df_train_x[0:4]

,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color,vehicle_age
381697,2004.0,28,0,6,2,151000.0,0,1,5,1,17.0
376451,2012.0,5,2,4,2,95000.0,0,1,9,8,9.0
325489,2008.0,3,0,6,2,171842.0,0,1,0,9,13.0
343894,2004.0,2,0,8,2,114000.0,0,2,9,10,17.0


In [9]:
df_test_x[0:4]

,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color,vehicle_age
99022,2016.0,2,0,8,2,46000.0,0,2,3,1,5.0
32902,2014.0,19,2,6,2,83788.0,0,2,0,0,7.0
406231,2014.0,7,2,6,2,95484.0,0,0,0,5,7.0
48019,2011.0,16,0,6,2,107500.0,0,0,0,5,10.0


#### **Training and Evaluation**

In [10]:
#Normalization and Label 
preprocessor = make_column_transformer(
    (OneHotEncoder(), ["manufacturer",'condition', 'cylinders', 'fuel', 'transmission', 'drive', 'type', 'paint_color' ]),
    (StandardScaler(), ["year","odometer","vehicle_age"]),
    remainder="passthrough"
)

In [11]:
#RandomSearch to find range of best super parameters quickly

def random_search(_model,_preprocessor,_param_dist,_n_iter,_cv):
    pipeline = make_pipeline(_preprocessor, _model)
    random = RandomizedSearchCV(pipeline, param_distributions=_param_dist, n_iter=_n_iter, cv=_cv, n_jobs=-1)
    random.fit(df_train_x, df_train_y)
    # Print the best hyperparameters and the corresponding score
    print("Best parameters: {}".format(random.best_params_))
    print("Best cross-validation score: {:.2f}".format(random.best_score_))
    return random

#GridSearch to find best super parameters precisely
def grid_search(_model,_preprocessor,_param_grid,_cv):
    # Define the Gradient Boosting regression model
    # Define the pipeline to preprocess the features and fit the model
    pipeline = make_pipeline(_preprocessor, _model)
    grid = GridSearchCV(pipeline, param_grid=_param_grid, cv=5, n_jobs=-1)
    grid.fit(df_train_x, df_train_y)
    # Print the best hyperparameters and the corresponding score
    print("Best parameters: {}".format(grid.best_params_))
    print("Best cross-validation score: {:.2f}".format(grid.best_score_))
    return grid

    #test the performance of model by setting parameters manually
def test_score(_model,_preprocessor):
    best_pipeline = make_pipeline(_preprocessor, _model)
    best_pipeline.fit(df_train_x, df_train_y)
    df_test_pred = best_pipeline.predict(df_test_x)
    rmse = np.sqrt(mean_squared_error(df_test_y, df_test_pred))
    mae = mean_absolute_error(df_test_y, df_test_pred)
    r2 = r2_score(df_test_y, df_test_pred)
    print("R2 score: {}".format(r2))
    print("RMSE: {}".format(rmse))
    print("MAE: {}".format(mae))

   #test the performace of model by getting parameters trained by Search function
def test_score_param(_grid):
    df_test_pred = _grid.best_estimator_.predict(df_test_x)
    rmse = np.sqrt(mean_squared_error(df_test_y, df_test_pred))
    mae = mean_absolute_error(df_test_y, df_test_pred)
    r2 = r2_score(df_test_y, df_test_pred)
    print("R2 score: {}".format(r2))
    print("RMSE: {}".format(rmse))
    print("MAE: {}".format(mae))

### GradientBoostingRegressor

In [14]:
from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)
param_random = {
         "gradientboostingregressor__n_estimators":[980],
        "gradientboostingregressor__max_depth": np.arange(1, 10),
         "gradientboostingregressor__learning_rate": np.logspace(-3, 0, 100),
         "gradientboostingregressor__subsample": np.linspace(0.6, 1.0, 10),
    }
param_grid = {
        "gradientboostingregressor__n_estimators": [50,200,500],
        "gradientboostingregressor__max_depth": [3,5,7],
        "gradientboostingregressor__learning_rate": [0.001,0.01,0.1,1],
        "gradientboostingregressor__subsample": [0.3,0.5,0.9],
    }
# grid_G=random_search(GradientBoostingRegressor(),preprocessor,param_random,10,5)
grid_G=grid_search(GradientBoostingRegressor(),preprocessor,param_grid,cv)
test_score_param(grid_G)

Best parameters: {'gradientboostingregressor__learning_rate': 0.1, 'gradientboostingregressor__max_depth': 7, 'gradientboostingregressor__n_estimators': 500, 'gradientboostingregressor__subsample': 0.9}
Best cross-validation score: 0.82
R2 score: 0.8214734502718747
RMSE: 6159.999597470852
MAE: 3686.1962746059808


### XGBRegressor

In [33]:
from xgboost import XGBRegressor
param_random_X={
    'xgbregressor__subsample': np.linspace(0.9, 1.0, 10),
    'xgbregressor__reg_lambda': np.linspace(0.8,0.9, num=10),
    'xgbregressor__reg_alpha': np.arange(40,70,10),
    'xgbregressor__n_estimators': np.arange(800, 900, 10),
    'xgbregressor__min_child_weight': np.arange(5,8),
    'xgbregressor__max_depth': [7],
    # 'xgbregressor__learning_rate': np.logspace(-1, 0, 100),
    'xgbregressor__learning_rate': np.linspace(0.3,0.5,num=100),
    'xgbregressor__gamma': np.linspace(0.0, 0.1, num=10),
    'xgbregressor__colsample_bytree': np.linspace(0.9, 1, 10),
}
param_grid_X={
    'xgbregressor__subsample': [1],
    'xgbregressor__reg_lambda':[0.8] ,
    'xgbregressor__reg_alpha': [50],
    'xgbregressor__n_estimators': [880],
    'xgbregressor__min_child_weight': [2],
    'xgbregressor__max_depth': [7],
    'xgbregressor__learning_rate': [0.41],
    'xgbregressor__gamma': [0.05],
    'xgbregressor__colsample_bytree': [0.95]
}
# grid_X=random_search(XGBRegressor(),preprocessor,param_random_X,30,5)
grid_X=grid_search(XGBRegressor(),preprocessor,param_grid_X)
test_score_param(grid_X)

Best parameters: {'xgbregressor__colsample_bytree': 0.95, 'xgbregressor__gamma': 0.05, 'xgbregressor__learning_rate': 0.41, 'xgbregressor__max_depth': 7, 'xgbregressor__min_child_weight': 2, 'xgbregressor__n_estimators': 880, 'xgbregressor__reg_alpha': 50, 'xgbregressor__reg_lambda': 0.8, 'xgbregressor__subsample': 1}
Best cross-validation score: 0.84
R2 score: 0.8398418059690647
RMSE: 5890.1250370049465
MAE: 3181.194474591849


### LinearRegression

In [16]:
param_grid_L={}
grid_L=grid_search(LinearRegression(),preprocessor,param_grid_L)
test_score_param(grid_L)

Best parameters: {}
Best cross-validation score: 0.48
R2 score: 0.47880117486908236
RMSE: 10470.395057481466
MAE: 6949.226828693397


### DecisionTree Regressor

In [13]:
from sklearn.tree import DecisionTreeRegressor

param_grid_D={
    'decisiontreeregressor__min_samples_leaf':np.arange(1,5),
    "decisiontreeregressor__min_weight_fraction_leaf": [0.0],
    'decisiontreeregressor__max_leaf_nodes':[None]
}
# grid_D=random_search(DecisionTreeRegressor(),preprocessor,param_grid_D,50,5)
grid_D=grid_search(DecisionTreeRegressor(),preprocessor,param_grid_D,5)
test_score_param(grid_D)

Best parameters: {'decisiontreeregressor__max_leaf_nodes': None, 'decisiontreeregressor__min_samples_leaf': 4, 'decisiontreeregressor__min_weight_fraction_leaf': 0.0}
Best cross-validation score: 0.79
R2 score: 0.7984547020204484
RMSE: 6545.08985539921
MAE: 3212.4455063189857


In [21]:
import multiprocessing

core_count = multiprocessing.cpu_count()
print("核心数:", core_count)

核心数: 8
